### Setup

In [4]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import yfinance as yf
from alpha_vantage.timeseries import TimeSeries
import time
import datetime
import matplotlib as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from code.functions import *
%matplotlib inline

from keys import client_id, secret_key, av_key, consumer_key

In [62]:
pd.set_option('display.max_columns',None)

### Alpha Vantage

In [2]:
ts = TimeSeries(key=av_key, output_format='pandas')
data, meta_data = ts.get_intraday(symbol='GME', interval='1min', outputsize='full')

In [ ]:
data

In [3]:
ticker = 'GME'

In [ ]:
csv_filepath = f'data/{ticker}_intra_Mar_15_26.csv'

In [ ]:
data.to_csv(csv_filepath, mode='w')  

In [4]:
csv_write(ticker, data)

### yfinance

In [5]:
gme_df = yf.download('GME', start='2021-02-03', end='2021-03-13', progress=False)
gme_df['Change'] = gme_df.Close - gme_df.Open
gme_df['Put/Call OI Ratio'] = float(0)

In [7]:
GME = yf.Ticker('GME')

In [11]:
opt = GME.option_chain('2021-04-01')

In [13]:
opt.calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,GME210401C00005000,2021-03-26 18:41:26,5.0,173.35,173.50,176.80,-0.949997,-0.545036,18.0,0,18.203129,True,REGULAR,USD
1,GME210401C00010000,2021-03-26 13:53:39,10.0,203.85,168.50,171.80,45.450012,28.693190,1.0,15,13.867189,True,REGULAR,USD
2,GME210401C00015000,2021-03-26 19:53:59,15.0,163.40,163.50,166.80,15.500000,10.480055,10.0,1,11.664065,True,REGULAR,USD
3,GME210401C00020000,2021-03-26 16:05:18,20.0,168.85,158.55,161.85,47.900010,39.603150,1.0,3,10.320316,True,REGULAR,USD
4,GME210401C00025000,2021-03-26 15:28:10,25.0,158.45,153.55,156.85,14.949997,10.418117,3.0,20,9.222660,True,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,GME210401C00650000,2021-03-26 19:59:51,650.0,1.02,0.85,1.25,-0.550000,-35.031850,272.0,232,5.041996,False,REGULAR,USD
203,GME210401C00660000,2021-03-26 19:11:49,660.0,0.95,0.69,1.27,-0.500000,-34.482760,38.0,47,5.039066,False,REGULAR,USD
204,GME210401C00670000,2021-03-26 19:59:51,670.0,1.01,0.71,1.01,-0.390000,-27.857143,53.0,344,4.994144,False,REGULAR,USD
205,GME210401C00680000,2021-03-26 19:56:29,680.0,0.83,0.80,0.90,-0.430000,-34.126984,233.0,220,5.031254,False,REGULAR,USD


### TDA API

In [5]:
# Import the client
from td.client import TDClient

# Create a new session, credentials path is required.
TDSession = TDClient(
    client_id= consumer_key,
    redirect_uri='https://github.com/eggrollofchaos',
    credentials_path='cred/td_state.json'
)

# Login to the session
TDSession.login()

# Grab real-time quotes for 'MSFT' (Microsoft)
msft_quotes = TDSession.get_quotes(instruments=['MSFT'])

# Grab real-time quotes for 'AMZN' (Amazon) and 'SQ' (Square)
multiple_quotes = TDSession.get_quotes(instruments=['AMZN','SQ'])

Please go to URL provided authorize your account: https://auth.tdameritrade.com/auth?response_type=code&redirect_uri=https%3A%2F%2Fgithub.com%2Feggrollofchaos&client_id=K8FL4RYE37I3J4DMAM59DVZCHSVYOCYY%40AMER.OAUTHAP
Paste the full URL redirect here: https://github.com/eggrollofchaos?code=FMWL0NIXNr%2Bf4732BGsmH9Up8Nd00%2FZcSXeojnXp35L5PRw4VABvSjnDpOw%2Bo%2FZRYqXVvuISSgr0at3bvKsCSgRC978GJNqlKjkc%2B%2BNfkTFw4Bf4HCQ1c1gc48OoV1fK90E8HxA8v%2FjQ6HJtcn99ZhBBxQzu%2BRY0kZV58QHSfvKBc%2BdID7FJB2Mvh3DEKjUZJNP8hyi%2BNSVZ5ZO8c4%2B3QDbirAVTzFsTeqFxHC%2FD6g6rv0kmIv9LKAXShfonrgFnUTKyaMssIgRTH%2BIhKGR%2BVTOAguk%2BVHDzpaL93jK7r%2FuBgmQ5xxnJfBWXrB4PnFTxN7v09JwTyWRaEoALF4lCoILvoF38DIkn1UAezjejdyw87Cvhmbp2ZaZ%2B%2B9kGUNL8lmz4A32sIqgEmlg4m83Co5rZ8%2FXx%2BfrJIp4fR3rL4OJKc9eSV61NVn4RXcr100MQuG4LYrgoVi%2FJHHvl%2B7W%2FMwshQVJepfHHCpcdA%2BQ5KpC5CKl6woaT%2BV94vnqn0didk3BUVXFrxs8lbBGHcwUpo5qA%2BgQ5757Gn%2FObTw12ixxo3uwPMVPu0AgbhOHZf9GuqZwuWKA%2Fyy3y59zkHZ07SLxSf23LNcEiwlvmm2JzVNptDTLJBld4CzNT4Pp6pemhPsu62%2BRSFviC

# EDA

In [6]:
# Login to the session
TDSession.login()

# Grab real-time quotes for 'MSFT' (Microsoft)
msft_quotes = TDSession.get_quotes(instruments=['MSFT'])

# Grab real-time quotes for 'AMZN' (Amazon) and 'SQ' (Square)
multiple_quotes = TDSession.get_quotes(instruments=['AMZN','SQ'])

In [8]:
gme_quotes = TDSession.get_quotes(instruments=['GME'])

In [116]:
gme_opt = TDSession.get_options_chain({'symbol':'GME',
                                      'fromDate':'2021-04-01',
                                      'toDate':'2021-04-01',
                                      'Digits':5})

In [117]:
gme_opt

{'symbol': 'GME',
 'status': 'SUCCESS',
 'underlying': None,
 'strategy': 'SINGLE',
 'interval': 0.0,
 'isDelayed': False,
 'isIndex': False,
 'interestRate': 0.1,
 'underlyingPrice': 178.575,
 'volatility': 29.0,
 'daysToExpiration': 0.0,
 'numberOfContracts': 414,
 'putExpDateMap': {'2021-04-01:4': {'5.0': [{'putCall': 'PUT',
     'symbol': 'GME_040121P5',
     'description': 'GME Apr 1 2021 5 Put (Weekly)',
     'exchangeName': 'OPR',
     'bid': 0.0,
     'ask': 0.01,
     'last': 0.01,
     'mark': 0.01,
     'bidSize': 0,
     'askSize': 1123,
     'bidAskSize': '0X1123',
     'lastSize': 0,
     'highPrice': 0.01,
     'lowPrice': 0.01,
     'openPrice': 0.0,
     'closePrice': 0.0,
     'totalVolume': 261,
     'tradeDate': None,
     'tradeTimeInLong': 1616783367957,
     'quoteTimeInLong': 1616785369837,
     'netChange': 0.01,
     'volatility': 642.844,
     'delta': 0.0,
     'gamma': 0.0,
     'theta': 0.0,
     'vega': 0.0,
     'rho': 0.0,
     'openInterest': 8980,
   

In [31]:
gme_dict = gme_opt['callExpDateMap']['2021-04-01:4']

In [54]:
gme_dict['5.0'][0].keys()

dict_keys(['putCall', 'symbol', 'description', 'exchangeName', 'bid', 'ask', 'last', 'mark', 'bidSize', 'askSize', 'bidAskSize', 'lastSize', 'highPrice', 'lowPrice', 'openPrice', 'closePrice', 'totalVolume', 'tradeDate', 'tradeTimeInLong', 'quoteTimeInLong', 'netChange', 'volatility', 'delta', 'gamma', 'theta', 'vega', 'rho', 'openInterest', 'timeValue', 'theoreticalOptionValue', 'theoreticalVolatility', 'optionDeliverablesList', 'strikePrice', 'expirationDate', 'daysToExpiration', 'expirationType', 'lastTradingDay', 'multiplier', 'settlementType', 'deliverableNote', 'isIndexOption', 'percentChange', 'markChange', 'markPercentChange', 'inTheMoney', 'mini', 'nonStandard'])

In [57]:
gme_c_df = pd.DataFrame.from_dict({i: gme_dict[i][0]
                        for i in gme_dict.keys()},
                       orient='index')

In [70]:
gme_c_df

,putCall,symbol,description,exchangeName,bid,ask,last,mark,bidSize,askSize,bidAskSize,lastSize,highPrice,lowPrice,openPrice,closePrice,totalVolume,tradeDate,tradeTimeInLong,quoteTimeInLong,netChange,volatility,delta,gamma,theta,vega,rho,openInterest,timeValue,theoreticalOptionValue,theoreticalVolatility,optionDeliverablesList,strikePrice,expirationDate,daysToExpiration,expirationType,lastTradingDay,multiplier,settlementType,deliverableNote,isIndexOption,percentChange,markChange,markPercentChange,inTheMoney,mini,nonStandard
5.0,CALL,GME_040121C5,GME Apr 1 2021 5 Call (Weekly),OPR,173.50,176.80,173.35,175.15,16,16,16X16,0,193.90,166.85,0.0,176.00,18,None,1616784086578,1616788795820,-2.65,642.844,1.000,0.000,0.000,0.000,0.000,0,-2.65,176.000,29.0,None,5.0,1617307200000,4,S,1617321600000,100.0,,,None,-1.51,-0.85,-0.48,True,False,False
10.0,CALL,GME_040121C10,GME Apr 1 2021 10 Call (Weekly),OPR,168.50,171.80,203.85,170.15,16,16,16X16,0,203.85,203.85,0.0,171.00,1,None,1616766819288,1616788795820,32.85,514.155,1.000,0.000,0.000,0.000,0.000,15,32.85,171.000,29.0,None,10.0,1617307200000,4,S,1617321600000,100.0,,,None,19.21,-0.85,-0.50,True,False,False
15.0,CALL,GME_040121C15,GME Apr 1 2021 15 Call (Weekly),OPR,163.50,166.80,163.40,165.15,16,16,16X16,0,180.00,163.40,0.0,166.00,10,None,1616788439743,1616788795819,-2.60,440.517,1.000,0.000,0.000,0.000,0.000,1,-2.60,166.000,29.0,None,15.0,1617307200000,4,S,1617321600000,100.0,,,None,-1.57,-0.85,-0.51,True,False,False
20.0,CALL,GME_040121C20,GME Apr 1 2021 20 Call (Weekly),OPR,158.55,161.85,168.85,160.20,16,16,16X16,0,168.85,168.85,0.0,161.00,1,None,1616774718801,1616788795822,7.85,423.924,1.000,0.000,0.000,0.000,0.000,3,7.85,161.000,29.0,None,20.0,1617307200000,4,S,1617321600000,100.0,,,None,4.88,-0.80,-0.50,True,False,False
25.0,CALL,GME_040121C25,GME Apr 1 2021 25 Call (Weekly),OPR,153.55,156.85,158.45,155.20,16,16,16X16,0,177.75,158.45,0.0,156.00,3,None,1616772490336,1616788795822,2.45,425.438,1.000,0.000,0.000,0.000,0.000,20,2.45,156.000,29.0,None,25.0,1617307200000,4,S,1617321600000,100.0,,,None,1.57,-0.80,-0.51,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650.0,CALL,GME_040121C650,GME Apr 1 2021 650 Call (Weekly),OPR,0.85,1.25,1.02,1.05,5,6,5X6,0,2.35,0.75,0.0,0.89,272,None,1616788791062,1616788791610,0.13,429.270,0.027,0.001,-0.480,0.015,0.001,232,1.02,0.891,29.0,None,650.0,1617307200000,4,S,1617321600000,100.0,,,None,14.44,0.16,17.81,False,False,False
660.0,CALL,GME_040121C660,GME Apr 1 2021 660 Call (Weekly),OPR,0.69,1.27,0.95,0.98,16,1,16X1,0,2.14,0.70,0.0,0.86,38,None,1616785909684,1616788795242,0.09,431.326,0.026,0.001,-0.469,0.015,0.001,47,0.95,0.863,29.0,None,660.0,1617307200000,4,S,1617321600000,100.0,,,None,10.13,0.12,13.61,False,False,False
670.0,CALL,GME_040121C670,GME Apr 1 2021 670 Call (Weekly),OPR,0.71,1.01,1.01,0.86,6,1,6X1,0,2.20,0.74,0.0,0.83,53,None,1616788791062,1616788799346,0.18,433.239,0.025,0.001,-0.458,0.014,0.001,344,1.01,0.834,29.0,None,670.0,1617307200000,4,S,1617321600000,100.0,,,None,21.13,0.03,3.14,False,False,False
680.0,CALL,GME_040121C680,GME Apr 1 2021 680 Call (Weekly),OPR,0.80,0.90,0.83,0.85,5,1,5X1,0,2.48,0.65,0.0,0.81,233,None,1616788589040,1616788797432,0.02,435.022,0.024,0.001,-0.447,0.014,0.001,220,0.83,0.805,29.0,None,680.0,1617307200000,4,S,1617321600000,100.0,,,None,3.09,0.04,5.58,False,False,False


In [64]:
gme_c_df.gamma.value_counts()

0.000    61
0.001    42
0.002    34
0.004    22
0.003    19
0.005    14
0.006    10
0.007     5
Name: gamma, dtype: int64

In [65]:
gme_c_df.delta.value_counts()

1.000    61
0.049     2
0.045     2
0.029     2
0.042     2
         ..
0.907     1
0.046     1
0.131     1
0.031     1
0.110     1
Name: delta, Length: 142, dtype: int64

In [123]:
def calc_gamma(strike, dte):
    S = 181
    K = strike
    std = 31.28
    t = dte/365
    rfr = 0.002163
    d1 = ( np.log(S/K) + (rfr + (std**2)/2)*t ) / (std * np.sqrt(t))
#     phi = np.exp( -1*(d1**2)/2 ) / np.sqrt((2*np.pi*t)
    return np.exp( -1*(d1**2)/2 ) / ( (S * std)*np.sqrt((2*np.pi*t)) )

In [120]:
# strikes = gme_c_df.strikePrice

In [121]:
gammas = calc_gamma(gme_c_df.strikePrice, 5)

In [122]:
gammas

5.0       0.000000e+00
10.0      0.000000e+00
15.0      0.000000e+00
20.0      0.000000e+00
25.0      0.000000e+00
             ...      
650.0    2.993610e-265
660.0    1.354574e-271
670.0    6.441482e-278
680.0    3.222481e-284
690.0    1.697509e-290
Name: strikePrice, Length: 207, dtype: float64

# Feature Engineering

GME open price
GME close price
SPY 
DIA
XRT close price
QQQ 
IWM
VXX
